<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# !pip install catboost

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import optuna

# Установка глобального рандом сида
random_seed = 42

In [2]:
path = r"E:\gnat\2\mm2.xlsx"
df = pd.read_excel(path)
df

,ID,Проекция,Сторона,"left, мм","top, мм","right, мм",condyle_width\n(ширина мыщелка),condyle_height\nвысота мыщелка),art_fossa_width\n(ширина ямки),art_fossa_depth\n(глубина ямки),Патология
0,100,Сагиттальная PR2,Левая,2.3,2.2,1.9,5.3,2.2,9.0,3.2,compression
1,102,Сагиттальная PR2,Левая,2.5,2.2,2.5,7.0,2.8,12.1,4.0,compression
2,108,Сагиттальная PR2,Левая,2.7,1.9,0.9,4.8,3.0,7.8,4.0,compression
3,109,Сагиттальная PR2,Левая,2.7,2.8,1.0,5.9,5.4,9.1,4.6,compression
4,101,Сагиттальная PR2,Левая,2.3,4.0,1.9,7.2,2.1,11.1,4.8,distal
5,105,Сагиттальная PR2,Левая,1.7,2.8,1.6,6.5,2.3,9.1,4.3,distal
6,106,Сагиттальная PR2,Левая,3.3,3.2,1.8,4.9,1.4,9.9,3.1,distal
7,118,Сагиттальная PR2,Левая,3.5,2.3,1.2,4.1,1.6,9.0,3.1,distal
8,107,Сагиттальная PR2,Левая,1.1,4.0,2.3,6.5,1.9,9.7,5.0,mesial
9,115,Сагиттальная PR2,Левая,1.1,5.6,5.7,5.6,1.0,11.3,4.5,mesial


In [3]:
columns_to_drop = ["ID", "Проекция", "Сторона"]
# columns_to_drop = ["ID", "Проекция", "Сторона",
#                   "condyle_width\n(ширина мыщелка)",
#                    "condyle_height\nвысота мыщелка)",
#                    "art_fossa_width\n(ширина ямки)",
#                    "art_fossa_depth\n(глубина ямки)"]
df = df.drop(columns=columns_to_drop)
df

,"left, мм","top, мм","right, мм",condyle_width\n(ширина мыщелка),condyle_height\nвысота мыщелка),art_fossa_width\n(ширина ямки),art_fossa_depth\n(глубина ямки),Патология
0,2.3,2.2,1.9,5.3,2.2,9.0,3.2,compression
1,2.5,2.2,2.5,7.0,2.8,12.1,4.0,compression
2,2.7,1.9,0.9,4.8,3.0,7.8,4.0,compression
3,2.7,2.8,1.0,5.9,5.4,9.1,4.6,compression
4,2.3,4.0,1.9,7.2,2.1,11.1,4.8,distal
5,1.7,2.8,1.6,6.5,2.3,9.1,4.3,distal
6,3.3,3.2,1.8,4.9,1.4,9.9,3.1,distal
7,3.5,2.3,1.2,4.1,1.6,9.0,3.1,distal
8,1.1,4.0,2.3,6.5,1.9,9.7,5.0,mesial
9,1.1,5.6,5.7,5.6,1.0,11.3,4.5,mesial


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   left, мм                        20 non-null     float64
 1   top, мм                         20 non-null     float64
 2   right, мм                       20 non-null     float64
 3   condyle_width
(ширина мыщелка)  20 non-null     float64
 4   condyle_height
высота мыщелка)  20 non-null     float64
 5   art_fossa_width
(ширина ямки)   20 non-null     float64
 6   art_fossa_depth
(глубина ямки)  20 non-null     float64
 7   Патология                       20 non-null     object 
dtypes: float64(7), object(1)
memory usage: 1.4+ KB


In [5]:
%%time 
# Wall time: 48.8 s

# Разделение на признаки и целевую переменную
X = df.drop('Патология', axis=1)
y = df['Патология']

# Отделение по одному объекту каждого класса для тестовой выборки
test_indices = df.groupby('Патология').apply(lambda x: x.sample(1, random_state=random_seed)).index.get_level_values(1)
train_indices = df.index.difference(test_indices)

X_train, X_test = X.loc[train_indices], X.loc[test_indices]
y_train, y_test = y.loc[train_indices], y.loc[test_indices]

# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def objective(trial):
    # Гиперпараметры для оптимизации
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0)
    }

    # Создание и обучение модели
    model = CatBoostClassifier(**param, random_seed=random_seed, verbose=0)
    model.fit(X_train, y_train)

    # Оценка модели
    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds, average='macro')  # Использование F1-меры
    return f1

# Создание и выполнение Optuna исследования
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Лучшие параметры
best_params = study.best_params
print('Лучшие параметры:', best_params)

# Обучение модели с лучшими параметрами
model = CatBoostClassifier(**best_params, random_seed=random_seed, verbose=0)
model.fit(X_train, y_train)

# Предсказания на тестовом наборе с лучшей моделью
predictions = model.predict(X_test)
predictions

[I 2023-12-20 23:35:17,153] A new study created in memory with name: no-name-18f1ba51-76bb-4189-92de-2ee32a9d18c5
[I 2023-12-20 23:35:17,604] Trial 0 finished with value: 0.4666666666666666 and parameters: {'iterations': 269, 'depth': 8, 'learning_rate': 0.1469722009924487, 'random_strength': 11, 'bagging_temperature': 0.6912206646392534, 'od_type': 'IncToDec', 'l2_leaf_reg': 3.796367038914624}. Best is trial 0 with value: 0.4666666666666666.
[I 2023-12-20 23:35:17,835] Trial 1 finished with value: 0.4666666666666666 and parameters: {'iterations': 417, 'depth': 4, 'learning_rate': 0.07561741765526572, 'random_strength': 4, 'bagging_temperature': 0.5398647066046969, 'od_type': 'IncToDec', 'l2_leaf_reg': 0.4197533653422736}. Best is trial 0 with value: 0.4666666666666666.
[I 2023-12-20 23:35:18,217] Trial 2 finished with value: 0.5333333333333333 and parameters: {'iterations': 577, 'depth': 5, 'learning_rate': 0.22599861004358293, 'random_strength': 8, 'bagging_temperature': 0.1964812544

Лучшие параметры: {'iterations': 241, 'depth': 4, 'learning_rate': 0.20528333387786188, 'random_strength': 8, 'bagging_temperature': 0.138509255696746, 'od_type': 'Iter', 'l2_leaf_reg': 2.741721687460305}
CPU times: total: 2min 2s
Wall time: 26.6 s


array([['compression'],
       ['compression'],
       ['distraction'],
       ['mesial'],
       ['norm']], dtype=object)

In [6]:
y_test

1     compression
5          distal
17    distraction
9          mesial
13           norm
Name: Патология, dtype: object

In [7]:
# from catboost import CatBoostClassifier
# import joblib

# # Сохранение модели в файл
# joblib.dump(model, 'catboost_model_mm_v3.pkl')

# # Для загрузки модели :
# # loaded_model = joblib.load('catboost_model.pkl')